In [1]:
from datetime import date

import polars as pl

In [2]:
DATA = '../data'
f = f'{DATA}/legislators-current.csv'

In [3]:
dtypes = {
    field: pl.Categorical
    for field in ['first_name', 'gender', 'type', 'state', 'party']}
df = (
    pl
    .read_csv(f, dtypes=dtypes)
    .with_column(pl.col('birthday').str.strptime(pl.Date, strict=False))
)
df.head()

last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
str,cat,str,str,str,str,date,cat,cat,cat,i64,i64,cat,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,i64,i64,i64,str,str,i64,str
"""Brown""","""Sherrod""",null,null,null,"""Sherrod Brown""",1952-11-09,"""M""","""sen""","""OH""",null,1,"""Democrat""","""https://www.br...","""503 Hart Senat...","""202-224-2315""","""https://www.br...","""http://www.bro...","""SenSherrodBrow...",43910797,"""SenatorSherrod...","""SherrodBrownOh...","""UCgy8jfERh-t_i...",null,"""B000944""",136,"""N00003535""","""S307""","""H2OH13033,S6OH...",5051,400050,27018,"""Sherrod Brown""",null,29389,"""Sherrod Brown"""
"""Cantwell""","""Maria""",null,null,null,"""Maria Cantwell...",1958-10-13,"""F""","""sen""","""WA""",null,1,"""Democrat""","""https://www.ca...","""511 Hart Senat...","""202-224-3441""","""https://www.ca...","""http://www.can...","""SenatorCantwel...",117501995,"""senatorcantwel...","""SenatorCantwel...","""UCN52UDqKgvHRk...",null,"""C000127""",172,"""N00007836""","""S275""","""S8WA00194,H2WA...",26137,300018,27122,"""Maria Cantwell...",null,39310,"""Maria Cantwell..."
"""Cardin""","""Benjamin""","""L.""",null,null,"""Benjamin L. Ca...",1943-10-05,"""M""","""sen""","""MD""",null,1,"""Democrat""","""https://www.ca...","""509 Hart Senat...","""202-224-4524""","""https://www.ca...","""http://www.car...","""SenatorCardin""",109071031,"""senatorbencard...","""senatorcardin""","""UCiQaJnMzlfzzG...",null,"""C000141""",174,"""N00001955""","""S308""","""H6MD03177,S6MD...",4004,400064,26888,"""Ben Cardin""",null,15408,"""Ben Cardin"""
"""Carper""","""Thomas""","""Richard""",null,null,"""Thomas R. Carp...",1947-01-23,"""M""","""sen""","""DE""",null,1,"""Democrat""","""https://www.ca...","""513 Hart Senat...","""202-224-2441""","""https://www.ca...","""http://www.car...","""SenatorCarper""",249787913,"""tomcarper""","""senatorcarper""","""UCgLnvbKwu4B3n...",null,"""C000174""",179,"""N00012508""","""S277""","""S8DE00079""",663,300019,22421,"""Tom Carper""",null,15015,"""Tom Carper"""
"""Casey""","""Robert""","""P.""","""Jr.""","""Bob""","""Robert P. Case...",1960-04-13,"""M""","""sen""","""PA""",null,1,"""Democrat""","""https://www.ca...","""393 Russell Se...","""202-224-6324""","""https://www.ca...","""http://www.cas...","""SenBobCasey""",171598736,"""SenatorBobCase...","""SenatorBobCase...","""UCtVssXhx-KuZa...",null,"""C001070""",1828,"""N00027503""","""S309""","""S6PA00217""",47036,412246,2541,"""Bob Casey, Jr....",null,40703,"""Bob Casey Jr."""


In [4]:
q = (
    df
    .lazy()
    .groupby('first_name')
    .agg([
        pl.count(),
        pl.col('gender').list(),
        pl.first('last_name')])
    .sort('count', reverse=True)
    .limit(10))
q.collect()

first_name,count,gender,last_name
cat,u32,list[cat],str
"""John""",21,"[""M"", ""M"", ... ""M""]","""Barrasso"""
"""Mike""",15,"[""M"", ""M"", ... ""M""]","""Kelly"""
"""Michael""",10,"[""M"", ""M"", ... ""M""]","""Bennet"""
"""Robert""",9,"[""M"", ""M"", ... ""M""]","""Casey"""
"""David""",8,"[""M"", ""M"", ... ""M""]","""Cicilline"""
"""Mark""",8,"[""M"", ""M"", ... ""M""]","""Warner"""
"""James""",7,"[""M"", ""M"", ... ""M""]","""Risch"""
"""Bill""",7,"[""M"", ""M"", ... ""M""]","""Cassidy"""
"""Dan""",5,"[""M"", ""M"", ... ""M""]","""Newhouse"""


In [5]:
q = (
    df
    .lazy()
    .groupby('state')
    .agg([
        (pl.col('party') == 'Democrat').sum().alias('dem'),
        (pl.col('party') == 'Republican').sum().alias('rep'),
        ((pl.col('party') != 'Democrat') 
         & (pl.col('party') != 'Rebublican')).sum().alias('other')])
    .sort('other', reverse=True)
    .limit(10))
q.collect()

state,dem,rep,other
cat,u32,u32,u32
"""TX""",13,27,27
"""FL""",8,22,22
"""CA""",42,12,12
"""NY""",17,11,11
"""OH""",6,11,11
"""TN""",1,10,10
"""NC""",7,9,9
"""GA""",7,9,9
"""IN""",2,9,9


In [6]:
q = (
    df
    .lazy()
    .groupby('state')
    .agg([
        (pl.col('party') == 'Democrat').sum().alias('dem'),
        (pl.col('party') == 'Republican').sum().alias('rep')])
    .sort('dem', reverse=True)
    .limit(10))
q.collect()

state,dem,rep
cat,u32,u32
"""CA""",42,12
"""NY""",17,11
"""IL""",16,3
"""TX""",13,27
"""NJ""",11,3
"""PA""",11,8
"""MA""",11,0
"""WA""",10,2
"""MI""",9,6


In [7]:
q = (
    df
    .lazy()
    .groupby(['state', 'party'])
    .agg([pl.count('party').alias('count')])
    .filter(
        (pl.col('party') == 'Democrat')
        | (pl.col('party') == 'Republican'))
    .sort('count', reverse=True)
    .limit(10))
q.collect()

state,party,count
cat,cat,u32
"""CA""","""Democrat""",42
"""TX""","""Republican""",27
"""FL""","""Republican""",22
"""NY""","""Democrat""",17
"""IL""","""Democrat""",16
"""TX""","""Democrat""",13
"""CA""","""Republican""",12
"""OH""","""Republican""",11
"""NY""","""Republican""",11


In [8]:
def compute_age() -> pl.Expr:
    return date(2021, 1, 1).year - pl.col('birthday').dt.year()

def avg_birthday(gender: str) -> pl.Expr:
    return (
        compute_age()
        .filter(pl.col('gender') == gender)
        .mean()
        .alias(f'mean {gender} birthday'))

In [9]:
q = (
    df
    .lazy()
    .groupby(['state'])
    .agg([
        avg_birthday('M'),
        avg_birthday('F'),
        (pl.col('gender') == 'M').sum().alias('n_male'),
        (pl.col('gender') == 'F').sum().alias('n_female')])
    .limit(10))
q.collect()

state,mean M birthday,mean F birthday,n_male,n_female
cat,f64,f64,u32,u32
"""AR""",58.833333,null,6,0
"""UT""",59.5,null,6,0
"""CA""",55.138889,65.833333,36,18
"""HI""",59.0,59.5,2,2
"""IN""",54.888889,44.0,9,2
"""ID""",66.75,null,4,0
"""WA""",56.25,55.75,4,8
"""MI""",61.0,51.571429,8,7
"""RI""",59.0,null,4,0


In [10]:
def get_person() -> pl.Expr:
    return pl.col('first_name') + pl.lit(' ') + pl.col('last_name')

In [11]:
q = (
    df
    .lazy()
    .sort('birthday', reverse=True)
    .groupby(['state'])
    .agg([
        get_person().first().alias('youngest'),
        get_person().last().alias('oldest')])
    .limit(10))
q.collect()

state,youngest,oldest
cat,str,str
"""NY""","""Alexandria Oca...","""Jerrold Nadler..."
"""TN""","""William Ogles""","""Steve Cohen"""
"""PR""","""Jenniffer Gonz...","""Jenniffer Gonz..."
"""VA""","""Abigail Spanbe...","""Robert Scott"""
"""FL""","""Maxwell Frost""","""Frederica Wils..."
"""ID""","""Russ Fulcher""","""James Risch"""
"""SC""","""Russell Fry""","""James Clyburn"""
"""IN""","""Rudy Yakym""","""James Baird"""
"""OR""","""Andrea Salinas...","""Earl Blumenaue..."


In [12]:
q = (
    df.
    lazy()
    .sort('birthday', reverse=True)
    .groupby(['state'])
    .agg([
        get_person().first().alias('youngest'),
        get_person().last().alias('eldest'),
        get_person().sort().first().alias('alphabetical')])
    .limit(10))
q.collect()

state,youngest,eldest,alphabetical
cat,str,str,str
"""IA""","""Ashley Hinson""","""Charles Grassl...","""Ashley Hinson"""
"""NH""","""Chris Pappas""","""Jeanne Shaheen...","""Ann Kuster"""
"""WY""","""Harriet Hagema...","""John Barrasso""","""Cynthia Lummis..."
"""RI""","""Seth Magaziner...","""John Reed""","""David Cicillin..."
"""AR""","""Tom Cotton""","""John Boozman""","""Bruce Westerma..."
"""FL""","""Maxwell Frost""","""Frederica Wils...","""Aaron Bean"""
"""UT""","""Blake Moore""","""Mitt Romney""","""Blake Moore"""
"""ND""","""Kelly Armstron...","""John Hoeven""","""John Hoeven"""
"""MI""","""Haley Stevens""","""Jack Bergman""","""Bill Huizenga"""
